In [ ]:
!pip install tensorflow matplotlib numpy scikit-learn seaborn nltk opencv-python

In [ ]:
# Implement the Continuous Bag of Words (CBOW) Model. Stages can be:
# a. Data preparation
# b. Generate training data
# c. Train model
# d. Output

In [0]:
# ============================================================
# Continuous Bag of Words (CBOW) Model Implementation
# Practical Exam Solution
# ============================================================

# a. Import required libraries
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import os
from sklearn.manifold import TSNE
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print(f"TensorFlow version: {tf.__version__}")

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ============================================================
# TEXT DOCUMENT DATASET
# ============================================================

print("=" * 60)
print("TEXT DOCUMENT DATASET FOR CBOW IMPLEMENTATION")
print("=" * 60)

# Sample textual document about Artificial Intelligence and Machine Learning
text_document = """
Artificial Intelligence and Machine Learning Fundamentals

Artificial intelligence is the simulation of human intelligence processes by computer systems. 
These processes include learning, reasoning, problem solving, perception, and linguistic understanding. 
Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from data.

Deep learning represents a specialized branch of machine learning that utilizes neural networks with multiple layers. 
These deep neural networks can automatically learn and make intelligent decisions from large amounts of data. 
Natural language processing enables computers to understand, interpret, and generate human language effectively.

Computer vision is another crucial field within artificial intelligence that allows computers to identify 
and process objects within digital images and videos. Reinforcement learning represents a machine learning 
paradigm where intelligent agents learn optimal behaviors through interaction with their environment.

Supervised learning involves training models using labeled datasets where each example contains input-output pairs. 
Unsupervised learning discovers hidden patterns and structures within unlabeled data. 
Semi-supervised learning combines both labeled and unlabeled data during the training process.

Neural networks consist of interconnected nodes organized in layers that process information. 
Each connection between nodes can transmit signals throughout the network. 
The learning process involves adjusting connection weights through backpropagation algorithms.

Convolutional neural networks demonstrate exceptional performance for computer vision tasks including image classification. 
Recurrent neural networks excel at processing sequential data like time series and natural language. 
Transformer architectures have revolutionized natural language processing through attention mechanisms.

Data preprocessing represents an essential step in machine learning pipelines. 
This includes data cleaning, handling missing values, feature scaling, and feature engineering. 
Feature selection techniques help identify the most relevant attributes for model training.

Model evaluation employs various metrics to assess performance. Classification tasks use accuracy, 
precision, recall, and F1-score while regression problems utilize mean squared error and R-squared values. 
Cross-validation ensures consistent model performance across different data partitions.

Overfitting occurs when models learn training data patterns too closely but fail to generalize to new data. 
Regularization techniques like L1 and L2 regularization help prevent overfitting by adding penalty terms.

The future development of artificial intelligence promises significant advancements across multiple industries 
including healthcare diagnostics, financial analysis, autonomous vehicles, and educational technology. 
Ethical considerations and responsible AI development remain critical as technology continues to evolve.

Machine learning algorithms require substantial computational resources and high-quality data. 
Training complex models demands significant processing power and memory capacity. 
Data quality directly impacts model performance and prediction accuracy.

Hyperparameter tuning optimizes model configuration settings including learning rates and network architecture. 
Techniques include grid search, random search, and Bayesian optimization methods. 
Proper hyperparameter selection significantly improves model performance.

Model deployment involves integrating trained models into production environments. 
Monitoring model performance over time remains essential as data distributions may change. 
Continuous learning systems can adapt to evolving data patterns automatically.
"""

print("📄 TEXT DOCUMENT LOADED SUCCESSFULLY!")
print(f"Document length: {len(text_document):,} characters")
print(f"Preview (first 500 characters):")
print(text_document[:500] + "..." if len(text_document) > 500 else text_document)

# ============================================================
# a. Data Preparation
# ============================================================

print("\n" + "=" * 60)
print("STAGE A: DATA PREPARATION")
print("=" * 60)

def preprocess_text(text, remove_stopwords=True, min_word_length=2):
    """
    Preprocess the text document for CBOW model training
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords if requested
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and len(token) >= min_word_length]
    else:
        tokens = [token for token in tokens if len(token) >= min_word_length]
    
    return tokens

# Preprocess the text document
print("🔄 PREPROCESSING TEXT DOCUMENT...")
tokens = preprocess_text(text_document, remove_stopwords=True, min_word_length=2)

print("📊 TEXT PROCESSING STATISTICS:")
print(f"Total tokens: {len(tokens):,}")
print(f"Unique tokens: {len(set(tokens)):,}")
print(f"Sample tokens (first 30): {tokens[:30]}")

# Build vocabulary
word_freq = Counter(tokens)
vocab = sorted(word_freq, key=word_freq.get, reverse=True)
word_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_word = {idx: word for idx, word in enumerate(vocab)}

vocab_size = len(vocab)
print(f"\n📚 VOCABULARY ANALYSIS:")
print(f"Vocabulary size: {vocab_size}")
print(f"Most common words (top 20):")
for word, freq in word_freq.most_common(20):
    print(f"  {word}: {freq}")

# Calculate vocabulary coverage
total_tokens = len(tokens)
covered_by_top_50 = sum(word_freq[word] for word in vocab[:50])
coverage_percentage = (covered_by_top_50 / total_tokens) * 100
print(f"Top 50 words cover {coverage_percentage:.1f}% of all tokens")

# ============================================================
# b. Generate Training Data
# ============================================================

print("\n" + "=" * 60)
print("STAGE B: GENERATE TRAINING DATA")
print("=" * 60)

def generate_cbow_data(tokens, word_to_index, window_size=2):
    """
    Generate training data for CBOW model
    """
    X = []  # Context words
    y = []  # Target words
    
    for i in range(window_size, len(tokens) - window_size):
        # Get context words (surrounding the target word)
        context_words = []
        for j in range(-window_size, window_size + 1):
            if j != 0:  # Skip the target word itself
                context_words.append(tokens[i + j])
        
        target_word = tokens[i]
        
        # Convert to indices
        context_indices = [word_to_index[word] for word in context_words]
        target_index = word_to_index[target_word]
        
        X.append(context_indices)
        y.append(target_index)
    
    return np.array(X), np.array(y)

# Generate training data
window_size = 2
print(f"🔄 GENERATING TRAINING DATA WITH WINDOW SIZE {window_size}...")
X, y = generate_cbow_data(tokens, word_to_index, window_size)

print("📊 TRAINING DATA GENERATED:")
print(f"Context words (X) shape: {X.shape}")
print(f"Target words (y) shape: {y.shape}")
print(f"Total training samples: {len(X):,}")
print(f"Context words per sample: {window_size * 2}")

# Display sample training examples
print(f"\n🔍 SAMPLE TRAINING EXAMPLES:")
print("Format: [left context] <TARGET> [right context]")
for i in range(min(5, len(X))):
    context_indices = X[i]
    target_index = y[i]
    
    left_context = [index_to_word[idx] for idx in context_indices[:window_size]]
    right_context = [index_to_word[idx] for idx in context_indices[window_size:]]
    target_word = index_to_word[target_index]
    
    print(f"Example {i+1}: {left_context} <{target_word}> {right_context}")

# ============================================================
# c. Train Model
# ============================================================

print("\n" + "=" * 60)
print("STAGE C: TRAIN CBOW MODEL")
print("=" * 60)

def build_cbow_model(vocab_size, embedding_dim=100, window_size=2):
    """
    Build the CBOW model architecture
    """
    model = models.Sequential([
        # Embedding layer for context words
        layers.Embedding(
            input_dim=vocab_size, 
            output_dim=embedding_dim, 
            input_length=window_size * 2,
            name='word_embeddings'
        ),
        
        # Average the embeddings of all context words
        layers.GlobalAveragePooling1D(name='context_averaging'),
        
        # Hidden layers
        layers.Dense(128, activation='relu', name='hidden_layer_1'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu', name='hidden_layer_2'),
        layers.Dropout(0.3),
        
        # Output layer with softmax activation
        layers.Dense(vocab_size, activation='softmax', name='output_layer')
    ])
    
    return model

# Model configuration
embedding_dim = 100
epochs = 100
batch_size = 32

print("🔧 BUILDING CBOW MODEL...")
cbow_model = build_cbow_model(vocab_size, embedding_dim, window_size)

print("📐 MODEL ARCHITECTURE:")
cbow_model.summary()

# Compile the model
cbow_model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✅ MODEL COMPILED SUCCESSFULLY")
print(f"Optimizer: Adam (learning_rate=0.001)")
print(f"Loss function: Sparse Categorical Crossentropy")
print(f"Embedding dimension: {embedding_dim}")
print(f"Training epochs: {epochs}")
print(f"Batch size: {batch_size}")

# Train the model
print(f"\n🚀 STARTING MODEL TRAINING...")
history = cbow_model.fit(
    X, y,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=1,
    shuffle=True
)

print("🎯 MODEL TRAINING COMPLETED!")

# ============================================================
# d. Output and Results
# ============================================================

print("\n" + "=" * 60)
print("STAGE D: OUTPUT AND RESULTS")
print("=" * 60)

# 1. Training Performance Visualization
plt.figure(figsize=(15, 5))

# Plot training history
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2, color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2, color='red')
plt.title('CBOW Model - Training and Validation Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, color='green')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, color='orange')
plt.title('CBOW Model - Training and Validation Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2. Extract Word Embeddings
embedding_layer = cbow_model.get_layer('word_embeddings')
word_embeddings = embedding_layer.get_weights()[0]

print(f"\n📊 WORD EMBEDDINGS EXTRACTED:")
print(f"Embedding matrix shape: {word_embeddings.shape}")
print(f"Total word vectors: {word_embeddings.shape[0]}")
print(f"Vector dimension: {word_embeddings.shape[1]}")

# 3. Word Similarity Analysis
def find_similar_words(query_word, embeddings, word_to_index, index_to_word, top_n=10):
    """
    Find words most similar to the query word using cosine similarity
    """
    if query_word not in word_to_index:
        return f"Word '{query_word}' not in vocabulary"
    
    query_idx = word_to_index[query_word]
    query_vec = embeddings[query_idx]
    
    similarities = {}
    for idx, vector in enumerate(embeddings):
        if idx != query_idx:
            # Calculate cosine similarity
            cosine_sim = np.dot(query_vec, vector) / (np.linalg.norm(query_vec) * np.linalg.norm(vector))
            similarities[index_to_word[idx]] = cosine_sim
    
    # Get top N most similar words
    most_similar = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return most_similar

print(f"\n🔍 SEMANTIC SIMILARITY ANALYSIS:")
test_words = ['learning', 'neural', 'data', 'model', 'training', 'intelligence']
for word in test_words:
    if word in word_to_index:
        similar_words = find_similar_words(word, word_embeddings, word_to_index, index_to_word, 8)
        print(f"\nWords similar to '{word}':")
        for similar_word, similarity in similar_words:
            print(f"  {similar_word}: {similarity:.4f}")

# 4. Word Embeddings Visualization
def visualize_embeddings(embeddings, word_freq, word_to_index, top_n=25):
    """
    Visualize word embeddings using t-SNE dimensionality reduction
    """
    # Get top N most frequent words
    top_words = [word for word, _ in word_freq.most_common(top_n)]
    top_indices = [word_to_index[word] for word in top_words]
    top_vectors = embeddings[top_indices]
    
    # Apply t-SNE for 2D visualization
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(10, top_n-1))
    vectors_2d = tsne.fit_transform(top_vectors)
    
    # Create visualization
    plt.figure(figsize=(14, 10))
    scatter = plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], alpha=0.7, s=80, 
                         c=range(top_n), cmap='tab20')
    
    # Add word labels
    for i, word in enumerate(top_words):
        plt.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), 
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=11, alpha=0.9,
                    bbox=dict(boxstyle="round,pad=0.3", fc="lightblue", alpha=0.7))
    
    plt.colorbar(scatter, label='Word Rank')
    plt.title(f'Word Embeddings Visualization - Top {top_n} Words', fontsize=16, fontweight='bold')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.grid(True, alpha=0.3)
    plt.show()

print(f"\n🎨 GENERATING WORD EMBEDDINGS VISUALIZATION...")
visualize_embeddings(word_embeddings, word_freq, word_to_index, 25)

# 5. Word Prediction Function
def predict_target_word(context_words, model, word_to_index, index_to_word, top_k=5):
    """
    Predict the target word given context words
    """
    # Convert context words to indices
    context_indices = []
    for word in context_words:
        if word in word_to_index:
            context_indices.append(word_to_index[word])
        else:
            return f"Word '{word}' not in vocabulary"
    
    # Reshape for model prediction
    context_array = np.array([context_indices])
    
    # Make prediction
    predictions = model.predict(context_array, verbose=0)[0]
    
    # Get top K predictions
    top_indices = np.argsort(predictions)[-top_k:][::-1]
    results = [(index_to_word[idx], predictions[idx]) for idx in top_indices]
    
    return results

print(f"\n🎯 WORD PREDICTION EXAMPLES:")
prediction_examples = [
    ['artificial', 'intelligence', 'machine', 'learning'],
    ['neural', 'networks', 'deep', 'learning'],
    ['data', 'processing', 'model', 'training'],
    ['supervised', 'unsupervised', 'learning', 'algorithms']
]

for i, context in enumerate(prediction_examples):
    if all(word in word_to_index for word in context):
        predictions = predict_target_word(context, cbow_model, word_to_index, index_to_word, 5)
        print(f"\nContext {i+1}: {context}")
        print("Top predictions:")
        for word, probability in predictions:
            print(f"  {word}: {probability:.4f}")
    else:
        print(f"\nContext {i+1}: {context}")
        print("  ❌ Some context words not in vocabulary")

# 6. Save Results to Files
def save_results(embeddings, word_to_index, word_freq, history):
    """
    Save all results to text files
    """
    # Save word embeddings
    with open('cbow_word_embeddings.txt', 'w', encoding='utf-8') as f:
        f.write(f"{embeddings.shape[0]} {embeddings.shape[1]}\n")
        for word, idx in word_to_index.items():
            vector_str = ' '.join(map(str, embeddings[idx]))
            f.write(f"{word} {vector_str}\n")
    
    # Save vocabulary with frequencies
    with open('cbow_vocabulary.txt', 'w', encoding='utf-8') as f:
        f.write("Word\tFrequency\n")
        f.write("-" * 20 + "\n")
        for word, freq in word_freq.most_common():
            f.write(f"{word}\t{freq}\n")
    
    # Save training history
    with open('cbow_training_history.txt', 'w', encoding='utf-8') as f:
        f.write("CBOW Model Training History\n")
        f.write("=" * 30 + "\n")
        f.write(f"Final Training Loss: {history.history['loss'][-1]:.4f}\n")
        f.write(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}\n")
        f.write(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}\n")
        f.write(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}\n")
        f.write(f"Vocabulary Size: {vocab_size}\n")
        f.write(f"Embedding Dimension: {embedding_dim}\n")
        f.write(f"Window Size: {window_size}\n")

print(f"\n💾 SAVING RESULTS TO FILES...")
save_results(word_embeddings, word_to_index, word_freq, history)
print("✅ Results saved to:")
print("   - cbow_word_embeddings.txt")
print("   - cbow_vocabulary.txt")
print("   - cbow_training_history.txt")

# 7. Final Evaluation Report
print(f"\n📈 FINAL EVALUATION REPORT")
print("=" * 50)
final_loss, final_accuracy = cbow_model.evaluate(X, y, verbose=0)
print(f"Training Loss: {final_loss:.4f}")
print(f"Training Accuracy: {final_accuracy:.4f}")
print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Vocabulary Size: {vocab_size}")
print(f"Training Samples: {len(X):,}")
print(f"Embedding Dimension: {embedding_dim}")

print(f"\n🔗 SEMANTIC RELATIONSHIPS CAPTURED:")
semantic_pairs = [
    ('learning', 'machine'),
    ('neural', 'network'),
    ('deep', 'learning'),
    ('data', 'training'),
    ('model', 'accuracy')
]

for word1, word2 in semantic_pairs:
    if word1 in word_to_index and word2 in word_to_index:
        vec1 = word_embeddings[word_to_index[word1]]
        vec2 = word_embeddings[word_to_index[word2]]
        similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        print(f"  {word1} ↔ {word2}: {similarity:.4f}")

print(f"\n🎉 CBOW MODEL IMPLEMENTATION COMPLETE!")
print("=" * 50)
print("✅ All stages completed successfully")
print("✅ Word embeddings learned and visualized")
print("✅ Semantic relationships captured")
print("✅ Model ready for NLP tasks")
print("✅ Results saved for future use")

TensorFlow version: 2.20.0
TEXT DOCUMENT DATASET FOR CBOW IMPLEMENTATION
📄 TEXT DOCUMENT LOADED SUCCESSFULLY!
Document length: 3,822 characters
Preview (first 500 characters):

Artificial Intelligence and Machine Learning Fundamentals

Artificial intelligence is the simulation of human intelligence processes by computer systems. 
These processes include learning, reasoning, problem solving, perception, and linguistic understanding. 
Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from data.

Deep learning represents a specialized branch of machine learning that utilizes neural networks with multiple layers. 
Thes...

STAGE A: DATA PREPARATION
🔄 PREPROCESSING TEXT DOCUMENT...
📊 TEXT PROCESSING STATISTICS:
Total tokens: 372
Unique tokens: 235
Sample tokens (first 30): ['artificial', 'intelligence', 'machine', 'learning', 'fundamentals', 'artificial', 'intelligence', 'simulation', 'human', 'intelligence', 'processes', 'computer', 'sy

C:\Users\Yoda ji\Favorites\timewaste\timewaste\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ word_embeddings (Embedding)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ context_averaging                    │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_1 (Dense)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_2 (Dense)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


✅ MODEL COMPILED SUCCESSFULLY
Optimizer: Adam (learning_rate=0.001)
Loss function: Sparse Categorical Crossentropy
Embedding dimension: 100
Training epochs: 100
Batch size: 32

🚀 STARTING MODEL TRAINING...
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.0000e+00 - loss: 5.4591 - val_accuracy: 0.0270 - val_loss: 5.4614
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0272 - loss: 5.4493 - val_accuracy: 0.0270 - val_loss: 5.4657
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0442 - loss: 5.4379 - val_accuracy: 0.0405 - val_loss: 5.4715
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0850 - loss: 5.4193 - val_accuracy: 0.0541 - val_loss: 5.4804
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0680 - loss: 5.3865 - val_accuracy: 0.0541 - val_loss: 5.4998
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0510 - loss: 5.3297 - val_accuracy: 0.0676 - val_loss: 5.5464
Epoch 7/100



KeyboardInterrupt


KeyboardInterrupt



In [ ]:
# ============================================================
# Continuous Bag of Words (CBOW) Model Implementation for CBOW.txt Final
# ============================================================

# a. Import required libraries
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
from sklearn.manifold import TSNE
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download NLTK data (run once)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print(f"TensorFlow version: {tf.__version__}")

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ============================================================
# a. Data Preparation
# ============================================================

print("=" * 60)
print("STAGE A: DATA PREPARATION")
print("=" * 60)

def load_and_preprocess_text_data():
    """
    Load and preprocess textual data from CBOW.txt file
    """
    try:
        # Load from CBOW.txt file
        with open('CBOW.txt', 'r', encoding='utf-8') as file:
            sample_text = file.read()
        
        print("📄 LOADED TEXT FROM CBOW.txt:")
        print(sample_text[:500] + "..." if len(sample_text) > 500 else sample_text)
        
        return sample_text
    except FileNotFoundError:
        print("❌ CBOW.txt file not found. Using sample text instead.")
        # Fallback sample text
        sample_text = """
        The speed of transmission is an important point of difference between the two viruses. 
        Influenza has a shorter median incubation period and a shorter serial interval than COVID-19 virus. 
        The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. 
        This means that influenza can spread faster than COVID-19.
        """
        return sample_text

def preprocess_text(text):
    """
    Preprocess the text: lowercase, remove punctuation, tokenize
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    
    return tokens

# Load and preprocess data
text_data = load_and_preprocess_text_data()
tokens = preprocess_text(text_data)

print(f"\n📊 TEXT STATISTICS:")
print(f"Total tokens: {len(tokens)}")
print(f"Unique tokens: {len(set(tokens))}")
print(f"First 20 tokens: {tokens[:20]}")

# Build vocabulary
word_freq = Counter(tokens)
vocab = sorted(word_freq, key=word_freq.get, reverse=True)
word_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_word = {idx: word for idx, word in enumerate(vocab)}

vocab_size = len(vocab)
print(f"\n📚 VOCABULARY:")
print(f"Vocabulary size: {vocab_size}")
print(f"Most common words: {word_freq.most_common(10)}")

# ============================================================
# b. Generate Training Data
# ============================================================

print("\n" + "=" * 60)
print("STAGE B: GENERATE TRAINING DATA")
print("=" * 60)

def generate_cbow_data(tokens, word_to_index, window_size=2):
    """
    Generate training data for CBOW model
    """
    X = []  # Context words
    y = []  # Target words
    
    for i in range(window_size, len(tokens) - window_size):
        # Get context words (surrounding words)
        context = []
        for j in range(-window_size, window_size + 1):
            if j != 0:  # Skip the target word itself
                context.append(tokens[i + j])
        
        # Get target word
        target = tokens[i]
        
        # Convert to indices
        context_indices = [word_to_index[word] for word in context]
        target_index = word_to_index[target]
        
        X.append(context_indices)
        y.append(target_index)
    
    return np.array(X), np.array(y)

# Generate training data
window_size = 2
X, y = generate_cbow_data(tokens, word_to_index, window_size)

print(f"📊 TRAINING DATA SHAPE:")
print(f"Context words (X): {X.shape}")
print(f"Target words (y): {y.shape}")
print(f"Window size: {window_size}")
print(f"Context words per sample: {X.shape[1]}")

# Display sample training data
print(f"\n🔍 SAMPLE TRAINING DATA:")
for i in range(5):
    context_words = [index_to_word[idx] for idx in X[i]]
    target_word = index_to_word[y[i]]
    print(f"Context: {context_words} -> Target: {target_word}")

# ============================================================
# c. Train Model
# ============================================================

print("\n" + "=" * 60)
print("STAGE C: TRAIN CBOW MODEL")
print("=" * 60)

def build_cbow_model(vocab_size, embedding_dim=50, window_size=2):
    """
    Build CBOW model architecture
    """
    model = models.Sequential([
        # Embedding layer for context words
        layers.Embedding(input_dim=vocab_size, 
                        output_dim=embedding_dim, 
                        input_length=window_size * 2,
                        name='embedding'),
        
        # Average the embeddings of context words
        layers.GlobalAveragePooling1D(name='average_pooling'),
        
        # Dense layer with ReLU activation
        layers.Dense(128, activation='relu', name='hidden_layer'),
        layers.Dropout(0.3),
        
        # Output layer with softmax for word prediction
        layers.Dense(vocab_size, activation='softmax', name='output')
    ])
    
    return model

# Model parameters
embedding_dim = 50
window_size = 2

# Build model
cbow_model = build_cbow_model(vocab_size, embedding_dim, window_size)

print("🔧 CBOW MODEL ARCHITECTURE:")
cbow_model.summary()

# Compile the model
cbow_model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✅ MODEL COMPILED:")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Loss: Sparse Categorical Crossentropy")
print(f"Metrics: Accuracy")

# Train the model
print("\n🚀 TRAINING CBOW MODEL...")
history = cbow_model.fit(
    X, y,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    shuffle=True
)

print("\n✅ Training completed!")

# ============================================================
# d. Output
# ============================================================

print("\n" + "=" * 60)
print("STAGE D: OUTPUT AND EVALUATION")
print("=" * 60)

# Training history visualization
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('CBOW Model Training Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.title('CBOW Model Training Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Extract word embeddings
embedding_layer = cbow_model.get_layer('embedding')
weights = embedding_layer.get_weights()[0]
print(f"\n📊 WORD EMBEDDINGS:")
print(f"Embedding matrix shape: {weights.shape}")

# Visualize word embeddings using t-SNE
def visualize_embeddings(weights, words, num_words=20):
    """
    Visualize word embeddings using t-SNE
    """
    # Get top N words
    top_words = words[:num_words]
    top_indices = [word_to_index[word] for word in top_words]
    top_embeddings = weights[top_indices]
    
    # Apply t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, num_words-1))
    embeddings_2d = tsne.fit_transform(top_embeddings)
    
    # Plot
    plt.figure(figsize=(12, 8))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, s=100)
    
    for i, word in enumerate(top_words):
        plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                    fontsize=12, alpha=0.8,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
    
    plt.title('Word Embeddings Visualization (t-SNE) - CBOW.txt', fontsize=16, fontweight='bold')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.grid(True, alpha=0.3)
    plt.show()

# Visualize top words
top_words = [word for word, _ in word_freq.most_common(20)]
visualize_embeddings(weights, top_words)

# Function to find similar words
def find_similar_words(word, weights, word_to_index, index_to_word, top_n=5):
    """
    Find similar words based on cosine similarity
    """
    if word not in word_to_index:
        return f"Word '{word}' not in vocabulary"
    
    # Get word vector
    word_idx = word_to_index[word]
    word_vec = weights[word_idx]
    
    # Calculate cosine similarities
    similarities = {}
    for idx, vec in enumerate(weights):
        if idx != word_idx:
            cosine_sim = np.dot(word_vec, vec) / (np.linalg.norm(word_vec) * np.linalg.norm(vec))
            similarities[index_to_word[idx]] = cosine_sim
    
    # Get top N similar words
    similar_words = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    return similar_words

# Test similar words
print("\n🔍 SIMILAR WORD ANALYSIS:")
test_words = ['virus', 'influenza', 'transmission', 'covid']
for word in test_words:
    if word in word_to_index:
        similar = find_similar_words(word, weights, word_to_index, index_to_word)
        print(f"Words similar to '{word}': {similar}")

# Word prediction function
def predict_target_word(context_words, model, word_to_index, index_to_word, top_k=3):
    """
    Predict target word given context words
    """
    # Convert context words to indices
    context_indices = [word_to_index[word] for word in context_words if word in word_to_index]
    
    if len(context_indices) != window_size * 2:
        return "Invalid context words"
    
    # Reshape for prediction
    context_array = np.array([context_indices])
    
    # Make prediction
    predictions = model.predict(context_array, verbose=0)[0]
    
    # Get top K predictions
    top_indices = np.argsort(predictions)[-top_k:][::-1]
    top_words = [(index_to_word[idx], predictions[idx]) for idx in top_indices]
    
    return top_words

# Test word prediction
print(f"\n🎯 WORD PREDICTION EXAMPLES:")
test_contexts = [
    ['influenza', 'virus', 'serial', 'interval'],
    ['covid', 'virus', 'transmission', 'speed'],
    ['incubation', 'period', 'shorter', 'median']
]

for context in test_contexts:
    predictions = predict_target_word(context, cbow_model, word_to_index, index_to_word)
    print(f"Context: {context} -> Predictions: {predictions}")

# Save word embeddings
def save_embeddings(weights, word_to_index, filename='cbow_embeddings.txt'):
    """
    Save word embeddings to file
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"{vocab_size} {embedding_dim}\n")
        for word, idx in word_to_index.items():
            vec = ' '.join(map(str, weights[idx]))
            f.write(f"{word} {vec}\n")
    print(f"✅ Word embeddings saved to {filename}")

save_embeddings(weights, word_to_index)

# Final model evaluation
print(f"\n📈 FINAL MODEL EVALUATION:")
final_loss, final_accuracy = cbow_model.evaluate(X, y, verbose=0)
print(f"Final Training Loss: {final_loss:.4f}")
print(f"Final Training Accuracy: {final_accuracy:.4f}")

# Display vocabulary statistics
print(f"\n📚 VOCABULARY STATISTICS:")
print(f"Total words in vocabulary: {vocab_size}")
print(f"Embedding dimension: {embedding_dim}")
print(f"Window size: {window_size}")
print(f"Training samples: {len(X)}")

# Show some learned relationships
print(f"\n🔗 LEARNED WORD RELATIONSHIPS:")
relationship_examples = [
    (['influenza', 'virus'], 'serial'),
    (['covid', 'virus'], 'transmission'),
    (['incubation', 'period'], 'shorter')
]

for context, expected in relationship_examples:
    if all(word in word_to_index for word in context):
        prediction = predict_target_word(context, cbow_model, word_to_index, index_to_word, top_k=1)
        if prediction and isinstance(prediction, list):
            print(f"Context: {context} | Expected: '{expected}' | Predicted: '{prediction[0][0]}'")

# Additional analysis: Show word frequencies
print(f"\n📊 WORD FREQUENCY ANALYSIS:")
print("Most frequent words in CBOW.txt:")
for word, freq in word_freq.most_common(15):
    print(f"  {word}: {freq} occurrences")

# Semantic relationships analysis
print(f"\n🔍 SEMANTIC RELATIONSHIP ANALYSIS:")
medical_terms = ['virus', 'transmission', 'influenza', 'covid', 'symptoms']
for term in medical_terms:
    if term in word_to_index:
        similar = find_similar_words(term, weights, word_to_index, index_to_word, top_n=3)
        print(f"'{term}' relates to: {similar}")

print("\n" + "=" * 70)
print("CBOW MODEL IMPLEMENTATION FOR CBOW.txt COMPLETE!")
print("=" * 70)